# Scrape and Clean Commuter Line Jabodetabek Schedule

By Denny Dewanta

This project aims to scrape the Commuter Line Jabodetabek schedule from KAI Commuter Line API. Actually the API is currently open for public, so everyone can get the schedule data without any requirement needed. The data scraping including data as of March 24th 2024.

## Import Libraries and Data

In [44]:
import requests
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


We still need a 'dictionary' to scrape the data by using the station_id. I just made a dimension table to be our 'dictionary' to call the data. The station_id and station_name information are gathered from some trusted websites (actually we can't get the station_id officially sadly).

In [45]:
dim_station = pd.read_csv('dim_station.csv')
dim_station.head()

,station_id,station_name
0,AC,ANCOL
1,AK,ANGKE
2,BJD,BOJONGGEDE
3,BKS,BEKASI
4,BOI,BOJONG INDAH


## Functions

In [46]:
def get_data(station_id, station_name):
  # URL of the API
  url = f'https://api-partner.krl.co.id/krlweb/v1/schedule?stationid={station_id}&timefrom=01:00&timeto=24:00'

  # Send a GET request to the API
  response = requests.get(url)

  # Initialize an empty list to store each row of data
  data_rows = []

  # Check if the request was successful
  if response.status_code == 200:
      # Parse the JSON response
      data = response.json()

      # Check if the status in the JSON response is 200
      if data['status'] == 200:
          # Loop through each item in the data list
          for item in data['data']:
              # Append each item (as a dict) to the data_rows list
              data_rows.append(item)

          # Create a DataFrame from the list of dicts
          df = pd.DataFrame(data_rows)
      else:
          print('Error: The response from the API was not successful.')
  else:
      print('Error: Failed to retrieve data from the API.')

  return df

## Data Scraping

In [47]:
# Create an empty DataFrame for collecting all results
df = pd.DataFrame()

# Create an empty DataFrame for collecting failed station attempts
failed_stations = pd.DataFrame(columns=['station_id', 'station_name'])

In [48]:
# First pass: Attempt to scrape data for all stations
for index, row in dim_station.iterrows():
    try:
        # Attempt to scrape data for the station
        station_data = get_data(row['station_id'], row['station_name'])

        # Add a new column to flag the station name
        station_data['station_departure_name'] = row['station_name']
        station_data['station_departure_id'] = row['station_id']

        # Append the results to the df DataFrame
        df = pd.concat([df, station_data], ignore_index=True)
    except Exception as e:
        # If scraping fails, append the station to the failed_stations DataFrame
        failed_stations = pd.concat([failed_stations, pd.DataFrame({'station_id': [row['station_id']],
                                                                    'station_name': [row['station_name']]})],
                                    ignore_index=True)

Error: Failed to retrieve data from the API.


In [49]:
# Second pass: Attempt to scrape data for stations that failed in the first pass
for index, row in failed_stations.iterrows():
    try:
        # Attempt to scrape data again for the station
        station_data = get_data(row['station_id'], row['station_name'])

        # Add a new column to flag the station name
        station_data['station_departure_name'] = row['station_name']
        station_data['station_departure_id'] = row['station_id']

        # Append the results to the df DataFrame
        df = pd.concat([df, station_data], ignore_index=True)
    except Exception as e:
        # If scraping fails again, print warning to check whether if the station_id wrong or anything
        print(f"Failed to scrape data for station ID {row['station_id']} on second attempt.")

Error: Failed to retrieve data from the API.
Failed to scrape data for station ID GMR on second attempt.


## Data Cleaning

In [50]:
df.rename(columns={'dest': 'station_destination_name'}, inplace=True)

In [51]:
# Find station_ids in dim_station that are not in all_stations_data
missing_station_ids = dim_station[~dim_station['station_id'].isin(df['station_departure_id'])]

# Print out the missing station_ids
if missing_station_ids.empty:
    print("All station data is complete.")
else:
    print("Missing data for station IDs:")
    print(missing_station_ids['station_id'].to_list())

Missing data for station IDs:
['GMR']


Because GAMBIR currently not serving commuter line anymore, we can skip that.

In [52]:
unique_destinations = df['station_destination_name'].unique()
print(unique_destinations)

['TANJUNGPRIUK' 'JAKARTAKOTA' 'KAMPUNGBANDAN' 'BEKASI' 'MANGGARAI'
 'TAMBUN' 'CIKARANG' 'BOGOR' 'DEPOK' 'ANGKE' 'DURI' 'TANGERANG'
 'BANDARASOEKARNOHATTA' 'TANAHABANG' 'PARUNGPANJANG' 'RANGKASBITUNG'
 'TIGARAKSA' 'SERPONG' 'NAMBO' 'BOJONGGEDE' 'TANAH ABANG' 'MERAK']


Now, we want to remove the KA Bandara & local train to MERAK schedules to ensure our data consisted of commuter line data only.

In [53]:
df = df[~df['station_destination_name'].isin(['BANDARASOEKARNOHATTA', 'MERAK'])]

Since, there is different naming for TANAHABANG, we can just follow the majority format that is not using space in station_destination_name. We can rename 'TANAH ABANG' to 'TANAHABANG'

In [54]:
df['station_destination_name'] = df['station_destination_name'].replace('TANAH ABANG', 'TANAHABANG')

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17073 entries, 0 to 17303
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   train_id                  17073 non-null  object
 1   ka_name                   17073 non-null  object
 2   route_name                17073 non-null  object
 3   station_destination_name  17073 non-null  object
 4   time_est                  17073 non-null  object
 5   color                     17073 non-null  object
 6   dest_time                 17073 non-null  object
 7   station_departure_name    17073 non-null  object
 8   station_departure_id      17073 non-null  object
dtypes: object(9)
memory usage: 1.3+ MB


In [56]:
# Perform the left join
df = pd.merge(left=df, right=dim_station, how='left', left_on='station_destination_name', right_on='station_name')

# Drop the redundant station_name column after the merge
df.drop('station_name', axis=1, inplace=True)

In [57]:
df.rename(columns={'station_id': 'station_destination_id'}, inplace=True)

Renaming the fields and add _utc7 just to ensure that the fields' name are consistent and giving you the timezone information as well. Anyway, at first I would like to use "arrival" to the dest_time but it sounds ambiguous :/

In [58]:
df.rename(columns={'time_est': 'departure_time_utc7'}, inplace=True)
df.rename(columns={'dest_time': 'destination_time_utc7'}, inplace=True)

Renaming the ka_name field because it sounds silly lol.

In [59]:
df.rename(columns={'ka_name': 'train_name'}, inplace=True)

In [60]:
df.head()

,train_id,train_name,route_name,station_destination_name,departure_time_utc7,color,destination_time_utc7,station_departure_name,station_departure_id,station_destination_id
0,2400,COMMUTER LINE TANJUNGPRIUK,JAKARTAKOTA-TANJUNGPRIUK,TANJUNGPRIUK,06:07:00,#DD0067,06:16:00,ANCOL,AC,TPK
1,2402,COMMUTER LINE TANJUNGPRIUK,JAKARTAKOTA-TANJUNGPRIUK,TANJUNGPRIUK,06:24:00,#DD0067,06:33:00,ANCOL,AC,TPK
2,2401,COMMUTER LINE TANJUNGPRIUK,TANJUNGPRIUK-JAKARTAKOTA,JAKARTAKOTA,06:34:00,#DD0067,06:42:00,ANCOL,AC,JAKK
3,2404,COMMUTER LINE TANJUNGPRIUK,JAKARTAKOTA-TANJUNGPRIUK,TANJUNGPRIUK,06:43:00,#DD0067,06:52:00,ANCOL,AC,TPK
4,2403,COMMUTER LINE TANJUNGPRIUK,TANJUNGPRIUK-JAKARTAKOTA,JAKARTAKOTA,06:50:00,#DD0067,06:58:00,ANCOL,AC,JAKK


In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17073 entries, 0 to 17072
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   train_id                  17073 non-null  object
 1   train_name                17073 non-null  object
 2   route_name                17073 non-null  object
 3   station_destination_name  17073 non-null  object
 4   departure_time_utc7       17073 non-null  object
 5   color                     17073 non-null  object
 6   destination_time_utc7     17073 non-null  object
 7   station_departure_name    17073 non-null  object
 8   station_departure_id      17073 non-null  object
 9   station_destination_id    17073 non-null  object
dtypes: object(10)
memory usage: 1.9+ MB


## Export Data

Since I'm using Google Colab, I store the data on my Google Drive. Feel free to modify this anyway.

In [62]:
df.to_csv('/content/drive/My Drive/commuter_line_schedule.csv', index=False)